# Debt Analysis - Version 1.0

In [ ]:
%%capture

import pandas as pd
import plotly.express as px
import aesoppy.aesop as aesop
import os

# Comon Variables
data = os.path.join(os.getcwd(), 'data')

# DataFrames
fin_df = pd.read_csv(os.path.join(data, 'financials.csv'), index_col='fiscal_year')
debt_df = aesop.Debt(financials_df=fin_df).debt_df


In [ ]:
debt_df.sort_index(ascending=False)

In [ ]:
# Reduce to 20 years
for index, row in debt_df.iterrows():
    if aesop.aesop_now.year - index >= 21:
        debt_df.drop(index, inplace=True)

In [ ]:
debt_df.sort_index(ascending=False)

## Current Ratio

In [ ]:
current_df = pd.DataFrame(debt_df[['current_assets', 'current_liabilities']], index=debt_df.index)
current_df['current_ratio'] = current_df['current_assets'] / current_df['current_liabilities']
current_df

In [ ]:
current_fig1 = px.line(current_df, x=current_df.index, y=['current_ratio'], color_discrete_sequence=['blue'])
current_fig1.show()

## Debt/Cash Coverage & Pay Down

In [ ]:
cash_df = pd.DataFrame(debt_df[['cash_equivalents', 'market_securities', 'treasury_stock', 'current_assets',
                               'current_liabilities', 'long_liabilities', 'short_debt']], index=debt_df.index)
cash_df['cash_on_hand'] = cash_df['cash_equivalents'] + cash_df['market_securities'] - cash_df['treasury_stock']
cash_df['short_debt_coverage'] =  cash_df['short_debt'] / cash_df['cash_on_hand']
cash_df['total_debt_coverage'] =  (cash_df['current_liabilities'] + cash_df['long_liabilities']) / cash_df['cash_on_hand']
cash_df

In [ ]:
cash_df = px.bar(cash_df, x=cash_df.index, y=['short_debt_coverage', 'total_debt_coverage'], barmode='group', color_discrete_sequence=['blue', 'red'])
cash_df.show()

## Debt/FCF

In [ ]:
fcf_df = pd.DataFrame(debt_df[['cash_from_ops', 'cash_for_dividends', 'free_cash_flow',
                               'current_liabilities', 'long_liabilities', 'short_debt']], index=debt_df.index)
fcf_df['adj_fcf'] = fcf_df['free_cash_flow'] - fcf_df['cash_for_dividends']
fcf_df['debt_fcf'] =  (fcf_df['current_liabilities'] + fcf_df['long_liabilities']) / fcf_df['adj_fcf']
fcf_df['debt_cfo'] =  (fcf_df['current_liabilities'] + fcf_df['long_liabilities']) / fcf_df['cash_from_ops']
fcf_df

## Debt/Revenue Margins & Growth

In [ ]:
rev_df = pd.DataFrame(debt_df[['revenue', 'current_liabilities', 'long_liabilities']], index=debt_df.index)
rev_df['debt_total'] = fcf_df['current_liabilities'] + fcf_df['long_liabilities']
rev_df['debt_margin'] = rev_df['debt_total'] / rev_df['revenue']
rev_df['debtgro'] = round(rev_df['debt_total'].pct_change() * 100, 2)
rev_df